# Single cell dynamics
Reproduces the cell-level figures using the shared dynamics module.

In [5]:
import sys
from pathlib import Path

def _find_repo_root(start: Path) -> Path:
    for p in [start, *start.parents]:
        if (p / 'clean').exists():
            return p
    return start

ROOT = _find_repo_root(Path(__file__).resolve()) if '__file__' in globals() else _find_repo_root(Path.cwd().resolve())
if str(ROOT) not in sys.path:
    sys.path.insert(0, str(ROOT))

import numpy as np
import matplotlib.pyplot as plt

from clean.lib import dynamics as dyn

FIG_DIR = Path('figures/cell')
FIG_DIR.mkdir(parents=True, exist_ok=True)


In [6]:
dt = 0.05
t_a, X_a, _ = dyn.simulate_rebound_modulate(T=10000, t_on=3000, t_off=3050, dt=dt, base_current=-1.5)
t_b, X_b, _ = dyn.simulate_rebound_modulate(T=10000, t_on=3000, t_off=3050, base_current=-1.5, dt=dt, modulation=-0.2)

start, end = int(2900 / dt), int(3300 / dt)
data_a = np.array(X_a)[start:end, :, 0]
data_b = np.array(X_b)[start:end, :, 0]
time = np.linspace(0, data_a.shape[0], data_a.shape[0]) * dt

plt.figure(figsize=(6, 4))
plt.plot(time, data_a, label='A')
plt.plot(time, data_b, label='B')
plt.ylabel('Voltage (mV)', fontsize=15)
plt.xlabel('Time (ms)', fontsize=15)
plt.tick_params(axis='both', labelsize=15)
plt.legend(fontsize=15)
plt.tight_layout()
plt.savefig(FIG_DIR / 'bursting_neuron_modulation.png', bbox_inches='tight')
plt.close()


In [7]:
dt = 0.05
t1, X1, _ = dyn.simulate_rebound(dyn.ss_Luka, dt=dt)
t2, X2, _ = dyn.simulate_rebound(dyn.ss_Luka, pulse_current=-3.5, dt=dt)
t3, X3, _ = dyn.simulate_rebound(dyn.ss_Luka_slow, dt=dt)

start, end = int(100 / dt), int(1000 / dt)
window = slice(start, end)

def _plot_trace(series, fname, show_legend=True):
    time = np.linspace(0, series[0][0].shape[0], series[0][0].shape[0]) * dt
    plt.figure(figsize=(6, 4))
    for values, lbl in series:
        plt.plot(time, values, label=lbl)
    plt.ylabel('(mV)', fontsize=15)
    plt.xlabel('(ms)', fontsize=15)
    plt.tick_params(axis='both', labelsize=15)
    if show_legend:
        plt.legend(fontsize=15)
    plt.tight_layout()
    plt.savefig(FIG_DIR / fname, bbox_inches='tight')
    plt.close()

_plot_trace([(np.array(X1)[window, :, 0], 'A'), (np.array(X2)[window, :, 0], 'B'), (np.array(X3)[window, :, 0], 'C')], 'spiking_neuron_3.png')
_plot_trace([(np.array(X1)[window, :, 0], 'A')], 'spiking_neuron_present.png', show_legend=False)
_plot_trace([(np.array(X3)[window, :, 0], 'C')], 'bursting_neuron_present.png', show_legend=False)


In [4]:
dt = 0.05
t1, X1, _ = dyn.simulate_rebound(dyn.ss_Luka_bursting, T=10000, t_on=3000, t_off=3800, dt=dt, base_current=-1.5)
t2, X2, _ = dyn.simulate_rebound(dyn.ss_Luka_bursting, T=10000, t_on=3000, t_off=3050, dt=dt, base_current=-1.5)

start, end = int(2500 / dt), int(5000 / dt)
window = slice(start, end)

plt.figure(figsize=(6, 4))
plt.plot(np.linspace(0, (window.stop - window.start), window.stop - window.start) * dt, np.array(X1)[window, :, 0], label='A')
plt.plot(np.linspace(0, (window.stop - window.start), window.stop - window.start) * dt, np.array(X2)[window, :, 0], label='B')
plt.ylabel('Voltage (mV)', fontsize=15)
plt.xlabel('Time (ms)', fontsize=15)
plt.tick_params(axis='both', labelsize=15)
plt.legend(fontsize=15)
plt.tight_layout()
plt.savefig(FIG_DIR / 'bursting_neuron_2.png', bbox_inches='tight')
plt.close()
